In [44]:
import icartt
import pandas as pd
import numpy as np
import datetime

In [32]:
#if not looping
filebeginning = "C:/Users/cphal/OneDrive/Desktop/Aerosols/Module C/Input/"
filename = filebeginning + "ACTIVATE-LARGE-OPTICAL_HU25_20220618_R2.ict"

In [3]:
#define instrument dictionaries
instruments = {'METADATA':['Hskping','MMS-1HZ','SUMMARY','MetNav','reveal','CAS'],
               'cloud_CPD':['CDP'],
               'cloud_CPSPD':['CPSPD'],
                'CCN':['CCN'],
                'LAS':['LAS'],
                'CNC':['CNC','MICROPHYSICAL','AerosolCloudConc'],
                'OPTICAL':['OPTICAL','EXT','ABS']
              }

campaigns = ['FIREX','KORUS','CAMP2EX','SEAC4RS','DC3','ACTIVATE','DISCOVER','NAAMES']

optical_variables = {'scat_blue' : ['totSC450_stdPT','Sc450_total','Scat450tot','drySC450_stdPT','Scat_blue_dry_TSI3563_LARGE'],
            'scat_green' : ['totSC550_stdPT','Sc550_total','Scat550tot','drySC550_stdPT','Scat_green_dry_TSI3563_LARGE'], 
            'scat_red' : ['totSC700_stdPT','Sc700_total','Scat700tot','drySC700_stdPT','Scat_red_dry_TSI3563_LARGE'],
            'abs_blue': ['Abs470_stdPT','Abs470_total','Abs470tot','Abs_blue_dry_PSAP_LARGE'],
            'abs_green' : ['Abs532_stdPT','Abs532_total','Abs532tot','Abs_green_dry_PSAP_LARGE'],
            'abs_red' : ['Abs660_stdPT','Abs660_total','Abs660tot','Abs_red_dry_PSAP_LARGE']
            }

meta_variables = {'lat' : ['Latitude','G_LAT'],
                  'long' : ['Longitude','G_LONG'],
                  'GPS_Altitude': ['MSL_GPS_Altitude','GPS_Altitude','G_ALT'],
                  'Pressure_Altitude': ['Pressure_Altitude',],
                  'Static_Air_Temp': ['Static_Air_Temp','T'],
                  'RH' : ['Relative_Humidity','Relative_Humidity_H2O','RHw'],
                  'Static_Pressure' : ['Static_Pressure','P']
                  }

cngt_variables = {'CNgt3nm':['CNgt3nm','CNgt3nm_stdPT'],
                  'CNgt10nm':['CNgt10nm','CNgt10nm_stdPT'],
                  'CNgt10nm_nonvol':['CNgt10nm_nonvol','CNgt10nm_nonvol_stdPT']}

lwc_variables = {'LWC' : ['LWC','LWC_CAS','LWC_gm3']}

stdpt = {'stdPT' : ['stdPT','stdpt']}

cloud_flag = {'Cloud_Flag': ['Cloud_Flag','CloudIndicator','Cloud']}

cloud_conc = {'cloud_number' : ['Number_Concentration_STP','N_CCN_stdPT']}


In [33]:
#Function purpose: find new filename
#Inputs:        filename: name of the file we are analyzing
#               instruments: list of the instruments
#               campaigns: list of the campaigns
#Returns:       tuple with campaign name and instrument name
def find_new_name(filename,instruments,campaigns):
    inst = ''
    camp = ''
    for key in instruments.keys():
        l = instruments.get(key)
        for word in l:
            if word in filename.upper(): inst = key
    if(len(inst)==0): print('ERROR: instrument not found')

    for c in campaigns:
        if c in filename.upper(): camp = c
    if(len(camp)==0): print('ERROR: campaign not found')

    return((camp,inst))

fileinfo = find_new_name(filename,instruments,campaigns)
newname = fileinfo[0]+'_'+fileinfo[1]+'_'

In [ ]:
#load in ict file to python
ict = icartt.Dataset(filename)
df = ict.data[:]

df = pd.DataFrame(df)

varnames = [x for x in ict.variables]

In [41]:
#example: looking at optical stuff
dictionary = optical_variables

#get the variables we want
columns_we_want = []
for x in varnames:
    for key in dictionary.keys():
        l = dictionary.get(key)
        for word in l:
            if word == x: 
                columns_we_want.append(key)
                df.rename(columns={word:key},inplace=True)
#we always want stdPT so add that too
if 'stdPT' in varnames:
    columns_we_want.append(varnames)

In [ ]:
#reformat date and time
date_time = ict.times
datetimedf = pd.DataFrame(date_time, columns= ['datetime'])
date = str(date_time[0])
date = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%f000').strftime('%Y_%m_%d')
datetimedf

In [ ]:
#create new dataframe
new_df = datetimedf
new_df[columns_we_want] = df[columns_we_want]